In [1]:
import numpy as np
import pandas as pd
import regex as re
from multiprocessing.pool import ThreadPool
import random
from scraper import StockInfo

In [24]:
ticker = "NVDA"
operatingMargin = np.array([35, 35, 37, 38, 39, 40, 40, 40, 40, 40, 40, 40]) / 100

RevenueGrowthRate = np.array([15, 15, 15, 15, 15, 12.72, 10.44, 8.16, 5.88, 3.60]) / 100
Costofcapital = np.array([12.21, 12.21, 12.21, 12.21, 12.21, 11.54, 10.86, 10.19, 9.52, 8.85]) / 100
TaxRate = np.array([10, 10, 10, 10, 10, 10, 13, 16, 19, 22, 25, 25]) / 100
salesToCapital = np.array([1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15])
forecastYears = 10
baseRevenue = 14028

In [30]:
salesToCapital

array([1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15])

In [59]:
class DCF:
    def __init__(self, ticker, operatingMargin, RevenueGrowthRate, TaxRate, Costofcapital, 
                 salesToCapital, baseRevenue, forecastYears = 10) -> None:
        self.ticker = ticker
        self.operatingMargin = operatingMargin
        self.RevenueGrowthRate = RevenueGrowthRate
        self.Costofcapital = Costofcapital
        self.TaxRate = TaxRate
        self.forecastYears = forecastYears
        self.baseRevenue = baseRevenue
        self.salesToCapital = salesToCapital
        
        self.CumulatedDiscountFactor = np.array([])
        self.OperatingIncomeForecast = np.array([])
        self.RevenueForecast = np.array([])
        self.EBITForecast = np.array([])
        self.ReinvestmentsForecast = np.array([])
        self.FCFF = np.array([])
        self.PV = np.array([])
    
    def make_CumulatedDiscountFactor(self):
        for i in range(self.forecastYears):
            if i == 0:
                self.CumulatedDiscountFactor = np.append(self.CumulatedDiscountFactor, 1 / (1 + self.Costofcapital[i]))
            else:
                self.CumulatedDiscountFactor = np.append(self.CumulatedDiscountFactor, self.Costofcapital[i - 1] * 1 / (1 + self.Costofcapital[i]))
                
        return self.CumulatedDiscountFactor
    
    def make_RevenueForecast(self):
        for i in range(self.forecastYears):
            if i == 0:
                self.RevenueForecast = np.append(self.RevenueForecast, self.baseRevenue * (self.RevenueGrowthRate[i] + 1))
            else:
                self.RevenueForecast = np.append(self.RevenueForecast, self.RevenueForecast[i - 1] * (self.RevenueGrowthRate[i] + 1))

        return self.RevenueForecast
    
    def make_OperatingIncomeForecast(self):
        if self.RevenueForecast.size <= 0:
            self.make_RevenueForecast()
        
        for i in range(self.forecastYears):
            self.OperatingIncomeForecast = np.append(self.OperatingIncomeForecast, self.operatingMargin[i] * self.RevenueForecast[i])

        return self.OperatingIncomeForecast
    
    def make_EBIT(self):
        if self.OperatingIncomeForecast.size <= 0:
            self.make_OperatingIncomeForecast()

        for i in range(self.forecastYears):
            self.EBITForecast = np.append(self.EBITForecast, self.OperatingIncomeForecast[i] * (1 - self.TaxRate[i]))    
        
        return self.EBITForecast
    
    def make_ReinvestmentsForecast(self):
        if self.RevenueForecast.size <= 0:
            self.make_RevenueForecast()

        for i in range(self.forecastYears):
            if i == 0:
                self.ReinvestmentsForecast = np.append(self.ReinvestmentsForecast, (self.RevenueForecast[i] - self.baseRevenue) / self.salesToCapital[i])   
            else:
                self.ReinvestmentsForecast = np.append(self.ReinvestmentsForecast, (self.RevenueForecast[i] - self.RevenueForecast[i - 1]) / self.salesToCapital[i]) 
        
        return self.ReinvestmentsForecast
    
    def make_FCFF(self):
        if self.EBITForecast.size <= 0 or self.RevenueForecast.size <= 0:
            self.make_ReinvestmentsForecast()
            self.make_EBIT()
            
        for i in range(self.forecastYears):
            self.FCFF = np.append(self.FCFF, self.EBITForecast[i] - self.ReinvestmentsForecast[i])
        
        return self.FCFF
    
    def make_PV(self):
        if self.FCFF.size <= 0 or self.CumulatedDiscountFactor.size <= 0:
            self.make_FCFF()
            self.make_CumulatedDiscountFactor()
        
        for i in range(self.forecastYears):
            self.PV = np.append(self.PV, self.FCFF[i] * self.CumulatedDiscountFactor[i])

        return self.PV

In [60]:
valuation = DCF(ticker, operatingMargin, RevenueGrowthRate, TaxRate, Costofcapital, salesToCapital, baseRevenue, forecastYears)
valuation.make_PV()

array([2898.05175079,  406.92993659,  509.75687852,  610.24819488,
        729.41737637,  911.72061082,  971.84470824, 1012.43711143,
       1031.86944664, 1027.01478967])

In [27]:
tick = StockInfo("ORCL")
cash = tick.get_BalanceSheet()
cash

AttributeError: 'StockInfo' object has no attribute 'make_ReinvestmentsForecast'